In [4]:
pip install evaluate datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00


In [5]:
import datasets
import transformers
import evaluate
import numpy as np
import torch # 파이토치

In [6]:
sms_spam = datasets.load_dataset('sms_spam')['train'].train_test_split(test_size=0.2, seed=42)
sms_spam

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})

In [7]:
sms_spam['train'].features['label'].names

['ham', 'spam']

# 데이터 전처리

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def preprocess_function(examples):
    return tokenizer(examples["sms"], truncation=True, padding=True, return_tensor=True)

In [10]:
sms_spam_preprocessed = sms_spam.map(lambda dct: tokenizer(dct["sms"], truncation=True))

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [12]:
# sms_spam_preprocessed['train'][3]


# 인공지능 생성

In [13]:
from transformers import AutoModelForSequenceClassification
인공지능 = model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 인공지능 학습

In [14]:
from transformers import DataCollatorWithPadding
데이터콜렉터 = datacollator = transformers.DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

In [15]:
import evaluate
import numpy as np
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2, # 전체문제세트를 2번 공부하라..
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sms_spam_preprocessed["train"],
    eval_dataset=sms_spam_preprocessed["test"],
    tokenizer=tokenizer,
    data_collator=datacollator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.045239,0.989238
2,0.056900,0.048296,0.991031


TrainOutput(global_step=558, training_loss=0.05233426653783381, metrics={'train_runtime': 78.2601, 'train_samples_per_second': 113.953, 'train_steps_per_second': 7.13, 'total_flos': 149317665725664.0, 'train_loss': 0.05233426653783381, 'epoch': 2.0})

# 예측하기

In [22]:
trainer.predict(sms_spam_preprocessed['train'])

PredictionOutput(predictions=array([[ 3.0996106, -3.1381652],
       [ 3.0302856, -3.008797 ],
       [ 3.1111367, -3.1558688],
       ...,
       [-2.5618117,  2.8672502],
       [ 2.6307523, -2.6514685],
       [ 2.9974263, -3.0544763]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.020852936431765556, 'test_accuracy': 0.9948418928010765, 'test_runtime': 10.0589, 'test_samples_per_second': 443.29, 'test_steps_per_second': 27.737})

In [24]:
trainer.predict(sms_spam_preprocessed['test'])

PredictionOutput(predictions=array([[-2.5901651,  2.8744745],
       [ 2.9593782, -3.0431693],
       [ 3.1413393, -3.184397 ],
       ...,
       [ 3.1529384, -3.2250335],
       [ 3.1483545, -3.2235978],
       [-2.576941 ,  2.8563967]], dtype=float32), label_ids=array([1, 0, 0, ..., 0, 0, 1]), metrics={'test_loss': 0.04523873329162598, 'test_accuracy': 0.989237668161435, 'test_runtime': 2.3875, 'test_samples_per_second': 467.011, 'test_steps_per_second': 29.319})

In [38]:
data = sms_spam_preprocessed['test'][::100]
type(data)

dict

In [46]:
data.keys()

dict_keys(['sms', 'label', 'input_ids', 'attention_mask'])

In [47]:
dataset = datasets.Dataset.from_dict(data)

In [48]:
전처리된나의데이터 = dataset.map(lambda dct: tokenizer(dct["sms"], truncation=True))

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [50]:
trainer.predict(전처리된나의데이터)

PredictionOutput(predictions=array([[-2.590165 ,  2.8744743],
       [ 3.0159504, -3.0616615],
       [ 2.866044 , -2.927828 ],
       [ 2.674664 , -2.6955123],
       [ 3.184652 , -3.264398 ],
       [ 3.123358 , -3.190426 ],
       [ 3.1414118, -3.2176175],
       [ 2.9639332, -3.091851 ],
       [ 2.8934278, -3.0066023],
       [ 3.0663908, -3.1958878],
       [ 3.0468876, -3.1529171],
       [ 2.931092 , -3.0008054]], dtype=float32), label_ids=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 0.002581640612334013, 'test_accuracy': 1.0, 'test_runtime': 0.0556, 'test_samples_per_second': 215.736, 'test_steps_per_second': 17.978})